In [4]:
import os
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from glob import glob
import re
import torch as T

In [5]:
BASE_PATH = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384"
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Using device: {DEVICE}")

🚀 Using device: cuda


In [7]:
class CropWeedDataset(Dataset):
    def __init__(self, base_dir):
        self.image_paths = []
        for subfolder in ['train_new', 'validation_new', 'test_new']:
            folder_path = os.path.join(base_dir, subfolder)
            self.image_paths.extend(glob(os.path.join(folder_path, "*.jpg")))
            self.image_paths.extend(glob(os.path.join(folder_path, "*.png")))

        def natural_key(string):
            return [int(s) if s.isdigit() else s for s in re.split(r'(\d+)', string)]
            
        self.image_paths.sort(key=natural_key)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = img.astype(np.float32) / 255.0
        img_tensor = torch.from_numpy(img).permute(2, 0, 1).contiguous()

        return img_tensor.to(DEVICE), img_path

def get_loader(base_path, batch_size=4):
    dataset = CropWeedDataset(base_path)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    return loader

loader = get_loader(BASE_PATH, batch_size=4)

for batch_imgs, _ in loader:
    print(f"✅ Batch shape: {batch_imgs.shape}")
    break

T.cuda.empty_cache()

✅ Batch shape: torch.Size([4, 3, 384, 512])


In [ ]:
def compute_fused_contours(batch_imgs):
    torch.cuda.empty_cache()
    fused_maps = []
    batch_imgs_np = batch_imgs.detach().cpu().numpy()

    for img in batch_imgs_np:
        img_rgb = np.transpose(img, (1, 2, 0)) * 255.0
        img_rgb = img_rgb.astype(np.uint8)
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)

        # Sobel
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
        sobel = cv2.magnitude(sobelx, sobely)
        sobel = np.clip(sobel / sobel.max(), 0, 1)

        # Canny
        canny = cv2.Canny(gray, 100, 200) / 255.0

        # Fuse Sobel + Canny
        fused = np.maximum(sobel, canny)

        fused_tensor = torch.tensor(fused, dtype=torch.float32).unsqueeze(0)
        fused_maps.append(fused_tensor)

    fused_batch = torch.stack(fused_maps).to(batch_imgs.device)
    torch.cuda.empty_cache()
    return fused_batch